In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [3]:
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[sql_query]
)

In [6]:
from langchain.messages import HumanMessage

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [7]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='af4f5e39-28d1-462a-bc92-c52eade2e270'),
 AIMessage(content='', additional_kwargs={'reasoning_content': "We need to query the database. We need to find the most popular artist whose name starts with 'S'. Likely there is a table of artists with popularity metric. We need to inspect schema. Let's query tables.", 'tool_calls': [{'id': 'fc_85624753-6011-4e31-b816-0b541b4cf238', 'function': {'arguments': '{"query":"SELECT name FROM sqlite_master WHERE type=\'table\';"}', 'name': 'sql_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 137, 'total_tokens': 218, 'completion_time': 0.167785475, 'completion_tokens_details': {'reasoning_tokens': 45}, 'prompt_time': 0.005772209, 'prompt_tokens_details': None, 'queue_time': 0.004803846, 'total_time': 0.173557684}, 'model_name': 'openai/gpt-oss-120b', 'sy

In [8]:
print(response["messages"][-3].tool_calls[0]['args']['query'])

SELECT ar.Name AS ArtistName, SUM(il.Quantity * il.UnitPrice) AS TotalSales
FROM Artist ar
JOIN Album al ON ar.ArtistId = al.ArtistId
JOIN Track tr ON al.AlbumId = tr.AlbumId
JOIN InvoiceLine il ON tr.TrackId = il.TrackId
JOIN Invoice i ON il.InvoiceId = i.InvoiceId
WHERE ar.Name LIKE 'S%'
GROUP BY ar.ArtistId
ORDER BY TotalSales DESC
LIMIT 1;
